In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
import joblib
from hsml.model_schema import ModelSchema
from hsml.schema import Schema
from sklearn.metrics import mean_absolute_error

import src.config as config
from src.data_utils import transform_ts_data_info_features_and_target
from src.inference import (
    fetch_days_data,
    get_hopsworks_project,
    load_metrics_from_registry,
    load_model_from_registry,
)
from src.pipeline_utils import get_pipeline

In [3]:
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
import hopsworks
import numpy as np
import pandas as pd
from hsfs.feature_store import FeatureStore

import src.config as config
from src.data_utils import transform_ts_data_info_features

In [4]:
def get_feature_store() -> FeatureStore:
    project = get_hopsworks_project()
    return project.get_feature_store()

In [5]:
def fetch_days_data(days):
    current_date = pd.to_datetime(datetime.now(timezone.utc)).tz_convert("America/New_York")
    fetch_data_from = current_date - timedelta(days=(days))
    fetch_data_to = current_date - timedelta(days=0)
    print(fetch_data_from, fetch_data_to)
    print("ghcgchcgfc")
    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_NAME, version=1)

    query = fg.select_all()
    # query = query.filter((fg.pickup_hour >= fetch_data_from))
    df = query.read()
    cond = (df["pickup_hour"] >= fetch_data_from) & (df["pickup_hour"] <= fetch_data_to)
    print(sum(cond))
    return df[cond]
    # return df

In [6]:
print(f"Fetching data from group store ...")
ts_data = fetch_days_data(180)

Fetching data from group store ...
2024-11-12 00:28:20.333393-05:00 2025-05-11 01:28:20.333393-04:00
ghcgchcgfc
2025-05-11 01:28:20,432 INFO: Initializing external client
2025-05-11 01:28:20,434 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 01:28:21,237 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215645
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.06s) 
366739


In [7]:
ts_data

,pickup_hour,pickup_location_id,rides
8,2025-03-25 19:00:00+00:00,JC066,0
11,2025-03-03 16:00:00+00:00,JC057,0
14,2024-12-17 03:00:00+00:00,JC066,0
17,2025-05-03 23:00:00+00:00,JC081,0
19,2025-03-11 07:00:00+00:00,JC034,0
...,...,...,...
746171,2025-03-03 13:00:00+00:00,JC052,0
746172,2025-04-20 19:00:00+00:00,HB506,0
746173,2024-11-29 00:00:00+00:00,JC009,4
746174,2024-11-19 21:00:00+00:00,JC107,0
